## Importing Libraries

In [1]:
import json
import requests
import pandas as pd
import numpy as np

## Importing Data From API and coverting it to DataFrame

In [5]:
r=requests.get(url="https://data.covid19india.org/v4/min/timeseries.min.json")
data=json.loads(r.content)
x=pd.DataFrame.transpose(pd.DataFrame(data))
x.reset_index(inplace=True)
x.head()

,index,dates
0,AN,"{'2020-03-26': {'delta': {'confirmed': 1}, 'de..."
1,AP,"{'2020-03-12': {'delta': {'confirmed': 1}, 'de..."
2,AR,"{'2020-04-02': {'delta': {'confirmed': 1}, 'de..."
3,AS,"{'2020-03-31': {'delta': {'confirmed': 1}, 'de..."
4,BR,"{'2020-03-22': {'delta': {'confirmed': 2}, 'de..."


## Extracting State and Date columns 

In [9]:
diii={}
for i in range(int(len(x))):
    key=(np.array(x)[i][0])
    value=list(dict(np.array(x)[i][1]).keys())
    
    diii[key]=value

stat=[]
dates=[]
for i,j in diii.items():
    for k in range(len(j)):
        stat.append(i)
        dates.append(j[k])
State_dates=pd.DataFrame({"State":stat,"dates":dates})
State_dates.head(10)

,State,dates
0,AN,2020-03-26
1,AN,2020-03-27
2,AN,2020-03-28
3,AN,2020-03-29
4,AN,2020-03-30
5,AN,2020-03-31
6,AN,2020-04-01
7,AN,2020-04-02
8,AN,2020-04-03
9,AN,2020-04-04


**Giving Sr.No to every entry to make unique records that will help for merging**

In [15]:
State_dates["Sr.No"]=np.arange(len(State_dates))

## For every Date Value Extracting delta delta7 total values and will join them with parent tabel (State_dates)

In [11]:
Concat_values=pd.DataFrame(x.loc[:,"dates"][0]).transpose()
for i in range(1,len(x.loc[:,"dates"])):
    first_st=pd.DataFrame(x.loc[:,"dates"][i]).transpose()
    Concat_values=pd.concat([Concat_values,first_st])
Concat_values.head(10)


,delta,delta7,total
2020-03-26,{'confirmed': 1},{'confirmed': 1},{'confirmed': 1}
2020-03-27,{'confirmed': 5},{'confirmed': 6},{'confirmed': 6}
2020-03-28,{'confirmed': 3},{'confirmed': 9},{'confirmed': 9}
2020-03-29,NaN,{'confirmed': 9},{'confirmed': 9}
2020-03-30,{'confirmed': 1},{'confirmed': 10},{'confirmed': 10}
2020-03-31,NaN,{'confirmed': 10},{'confirmed': 10}
2020-04-01,NaN,{'confirmed': 10},{'confirmed': 10}
2020-04-02,NaN,{'confirmed': 9},{'confirmed': 10}
2020-04-03,NaN,{'confirmed': 4},{'confirmed': 10}
2020-04-04,NaN,{'confirmed': 1},{'confirmed': 10}


In [12]:
## Adjusting Columns Of Concat_values and giving Sr.no to join them
Concat_values.reset_index(inplace=True)
Concat_values.columns=["Date","delta","delta7","total"]
Concat_values.insert(0,"Sr.No",np.arange(len(Concat_values)))


In [13]:
Concat_values.head(10)

,Sr.No,Date,delta,delta7,total
0,0,2020-03-26,{'confirmed': 1},{'confirmed': 1},{'confirmed': 1}
1,1,2020-03-27,{'confirmed': 5},{'confirmed': 6},{'confirmed': 6}
2,2,2020-03-28,{'confirmed': 3},{'confirmed': 9},{'confirmed': 9}
3,3,2020-03-29,NaN,{'confirmed': 9},{'confirmed': 9}
4,4,2020-03-30,{'confirmed': 1},{'confirmed': 10},{'confirmed': 10}
5,5,2020-03-31,NaN,{'confirmed': 10},{'confirmed': 10}
6,6,2020-04-01,NaN,{'confirmed': 10},{'confirmed': 10}
7,7,2020-04-02,NaN,{'confirmed': 9},{'confirmed': 10}
8,8,2020-04-03,NaN,{'confirmed': 4},{'confirmed': 10}
9,9,2020-04-04,NaN,{'confirmed': 1},{'confirmed': 10}


In [16]:
State_dates.head(10)

,State,dates,Sr.No
0,AN,2020-03-26,0
1,AN,2020-03-27,1
2,AN,2020-03-28,2
3,AN,2020-03-29,3
4,AN,2020-03-30,4
5,AN,2020-03-31,5
6,AN,2020-04-01,6
7,AN,2020-04-02,7
8,AN,2020-04-03,8
9,AN,2020-04-04,9


## From Extracted Data now we will further extract Delta and Other Values

## Extraction of  Delta Delta Values

In [17]:
##With Help of Key value of dictionaries taking Sr.NO and Delta sub Dictionaries and coverting them into dataFrame.
n=Concat_values.loc[:,["Sr.No","delta"]]
vv=np.array(n)
DI={}
for i in range(len(vv)):
    DI[vv[i][0]]=vv[i][1]

In [18]:
delta=pd.DataFrame(DI).transpose()
delta.insert(0,"Sr.No",np.arange(len(delta)))

delta

,Sr.No,confirmed,recovered,tested,other,deceased,vaccinated1,vaccinated2
0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
21673,21673,976.0,837.0,43322.0,NaN,15.0,323292.0,229056.0
21674,21674,990.0,845.0,45437.0,NaN,9.0,532317.0,287998.0
21675,21675,982.0,860.0,49017.0,NaN,8.0,701727.0,317576.0
21676,21676,980.0,880.0,47131.0,NaN,13.0,756507.0,304433.0


**Merging Delta table with our Parent table , Sr.No being same columns**

In [19]:
Stat_delta=State_dates.merge(delta,on="Sr.No",how="inner")
Stat_delta.columns=['State', 'dates', 'Sr.No', 'deltaconfirmed', 'deltarecovered', 'deltatested', 'deltaother',
       'deltadeceased', 'deltavaccinated1', 'deltavaccinated2']

In [20]:
Stat_delta.head(10)

,State,dates,Sr.No,deltaconfirmed,deltarecovered,deltatested,deltaother,deltadeceased,deltavaccinated1,deltavaccinated2
0,AN,2020-03-26,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,AN,2020-03-27,1,5.0,NaN,NaN,NaN,NaN,NaN,NaN
2,AN,2020-03-28,2,3.0,NaN,NaN,NaN,NaN,NaN,NaN
3,AN,2020-03-29,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AN,2020-03-30,4,1.0,NaN,NaN,NaN,NaN,NaN,NaN
5,AN,2020-03-31,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AN,2020-04-01,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AN,2020-04-02,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AN,2020-04-03,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AN,2020-04-04,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Extracting Delta7

In [21]:
n=Concat_values.loc[:,["Sr.No","delta7"]]
vv=np.array(n)
DI={}
for i in range(len(vv)):
    DI[vv[i][0]]=vv[i][1]
delta7=pd.DataFrame(DI).transpose()
delta7.columns=['delta7confirmed', 'delta7recovered', 'delta7tested', 'delta7other', 'delta7deceased', 'delta7vaccinated1',
       'delta7vaccinated2']

**Merging Delta7 table with our Extracted table , Sr.No being same columns**

In [22]:
delta7.insert(0,"Sr.No",np.arange(len(delta7)))
State_del_del7=Stat_delta.merge(delta7,on="Sr.No",how="inner")

## Extracting Total Column

In [25]:
n=Concat_values.loc[:,["Sr.No","total"]]
vv=np.array(n)
DI={}
for i in range(len(vv)):
    DI[vv[i][0]]=vv[i][1]
total=pd.DataFrame(DI).transpose()
total.insert(0,"Sr.No",np.arange(len(total)))
total.columns=['Sr.No', 'totalconfirmed', 'totalrecovered', 'totaltested', 'totalother', 'totaldeceased',
       'totalvaccinated1', 'totalvaccinated2']

**Merging Delta7 table with our Extracted table , Sr.No being same columns. This Will be our Final Table**

In [26]:
final=State_del_del7.merge(total,how="inner",on="Sr.No")

In [28]:
final.head(10)

,State,dates,Sr.No,deltaconfirmed,deltarecovered,deltatested,deltaother,deltadeceased,deltavaccinated1,deltavaccinated2,...,delta7deceased,delta7vaccinated1,delta7vaccinated2,totalconfirmed,totalrecovered,totaltested,totalother,totaldeceased,totalvaccinated1,totalvaccinated2
0,AN,2020-03-26,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,AN,2020-03-27,1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN
2,AN,2020-03-28,2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN
3,AN,2020-03-29,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN
4,AN,2020-03-30,4,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN
5,AN,2020-03-31,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN
6,AN,2020-04-01,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN
7,AN,2020-04-02,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN
8,AN,2020-04-03,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN
9,AN,2020-04-04,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN


"delta":{"confirmed":914,"deceased":15,"recovered":913,"tested":47417,"vaccinated1":209609,"vaccinated2":58881},"delta7":{"confirmed":6453,"deceased":86,"recovered":5953,"tested":300114,"vaccinated1":3809597,"vaccinated2":1871612},"total":{"confirmed":1592908,"deceased":19141,"recovered":1565471,"tested":19228303,"vaccinated1":56192166,"vaccinated2":21559747

In [36]:
#final.to_csv(r"C:\Users\91940\Downloads\time_final.csv")